# TODO
- Create a processing job that does nothing but create a print() output.
- Call it from the notebook.
- Pass in a parameter to the processing job from the notebook and use the results in the print() stateement.
(Now you can launch the processing job)

- Create a small notebook (checkout the examples in article 5) that loads a model and evaluates a model, as uses some test data (you can see getting the testdata in our Reddit notebook)
(Now you can load and use a model directly)
- And with the metrics I don’t know where we are. What we can use from experiments. If nothing we go for what we do today in the Reddit noteb

## References
* [1] https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_processing/scikit_learn_data_processing_and_model_evaluation/scikit_learn_data_processing_and_model_evaluation.ipynb
* [2] https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_processing/scikit_learn_data_processing_and_model_evaluation/scikit_learn_data_processing_and_model_evaluation.html
* [3] This one is for BYOC for running external scripts (preferred): https://docs.aws.amazon.com/sagemaker/latest/dg/processing-container-run-scripts.html
* [4] This one is for BYOC including code: https://docs.aws.amazon.com/sagemaker/latest/dg/build-your-own-processing-container.html

# 1. SK LEARN Hello World Sample

In [6]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor


region = boto3.session.Session().region_name

role = get_execution_role()

In [7]:
from sagemaker.sklearn.processing import SKLearnProcessor

sklearn_processor = SKLearnProcessor(
    framework_version="0.20.0",
    role=role,
    instance_type="ml.m5.xlarge",
    instance_count=1,
)

In [8]:
%%writefile processing.py

import argparse
import os
import warnings

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer, KBinsDiscretizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer

from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)


def print_shape():
    print('Data shape: {}, {} positive examples, {} negative examples')

if __name__=='__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--train-test-split-ratio', type=float, default=0.3)
    parser.add_argument('--dummy-argument', type=str, default="hello somebody")
    args, _ = parser.parse_known_args()
    
    print('Received arguments {}'.format(args))

    input_data_path = os.path.join('/opt/ml/processing/input', 'lorem.csv')
    print('Reading input data from {}'.format(input_data_path))
    
    
    print('Train data shape after preprocessing: {}')    
    train_features_output_path = os.path.join('/opt/ml/processing/train', 'lorem.csv')
    
    print('Saving training features to {}')


Writing processing.py


In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(
    code="processing.py",
    inputs=[
        ProcessingInput(source="s3://akirmak-r/text_classification.yaml", destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train_data", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/test"),
    ],
    arguments=["--train-test-split-ratio", "0.2", "--dummy-argument", "hello NLP article team"],
)

preprocessing_job_description = sklearn_processor.jobs[-1].describe()

## 2. PYTORCH Evaluation HELLO WORLD SAMPLE

## Build Container 

IMPORTANT: Run this from outside SM Studio)

In [1]:
# ## IMPORTANT: Run this from outside SM Studio)

# import boto3

# account_id = boto3.client('sts').get_caller_identity().get('Account')
# region = boto3.Session().region_name
# ecr_repository = 'akirmak-smprocessing'
# tag = ':latest'
# processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

# # Create ECR repository and push docker image
# !docker build -t $ecr_repository docker
# !aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
# !aws ecr create-repository --repository-name $ecr_repository
# !docker tag {ecr_repository + tag} $processing_repository_uri
# !docker push $processing_repository_uri

# processing_repository_uri

In [42]:
%%writefile Dockerfile

FROM 763104351884.dkr.ecr.eu-west-1.amazonaws.com/pytorch-inference:1.8.1-cpu-py36-ubuntu18.04
#FROM python:3.7-slim-buster

#RUN pip3 install pytorch

RUN pip3 install sagemaker sagemaker-experiments transformers nltk tensorboard sklearn jsonlines boto3
# Add a Python script and configure Docker to run it
ENTRYPOINT ["python3", "inference.py"]

Writing Dockerfile


In [44]:
%%writefile build_container.sh
#!/usr/bin/env bash

# Run from Terminal, e.g. from Cloud9
docker build -t 'akirmak-smprocessing' docker 
aws ecr get-login-password --region eu-west-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.eu-west-1.amazonaws.com
aws ecr get-login-password --region eu-west-1 | docker login --username AWS --password-stdin 123456789012.dkr.ecr.eu-west-1.amazonaws.com
aws ecr create-repository --repository-name 'akirmak-smprocessing'
docker tag akirmak-smprocessing:latest '123456789012.dkr.ecr.eu-west-1.amazonaws.com/akirmak-smprocessing:latest'
docker push '123456789012.dkr.ecr.eu-west-1.amazonaws.com/akirmak-smprocessing:latest'

Writing build_container.sh


In [30]:
# account_id = boto3.client('sts').get_caller_identity().get('Account')
# region = boto3.Session().region_name
# ecr_repository = 'akirmak-smprocessing'
# tag = ':latest'
# processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

In [36]:
# %%writefile build_container-AUto.sh
# #!/usr/bin/env bash

# # Run from Terminal, e.g. from Cloud9
# # Create ECR repository and push docker image
# docker build -t {ecr_repository} docker
# aws ecr get-login-password --region eu-west-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.eu-west-1.amazonaws.com
# aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
# aws ecr create-repository --repository-name $ecr_repository
# docker tag {ecr_repository + tag} $processing_repository_uri
# docker push $processing_repository_uri

Overwriting build_container-AUto.sh


In [14]:
%%writefile inference.py

import argparse
import os
import warnings

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer, KBinsDiscretizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer

from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)


def print_shape():
    print('Data shape: {}, {} positive examples, {} negative examples')

if __name__=='__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--train-test-split-ratio', type=float, default=0.3)
    parser.add_argument('--dummy-argument', type=str, default="hello somebody")
    args, _ = parser.parse_known_args()
    
    print('THIS IS **********INFERENCE FROM PYTORCH****************************** CONTAINER')
    print('Received arguments {}'.format(args))

    input_data_path = os.path.join('/opt/ml/processing/input', 'lorem.csv')
    print('Reading input data from {}'.format(input_data_path))
    
    
    print('Train data shape after preprocessing: {}')    
    train_features_output_path = os.path.join('/opt/ml/processing/train', 'lorem.csv')
    
    print('Saving training features to {}')


Writing inference.py


In [48]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

script_processor = ScriptProcessor(command=['python3'],
                image_uri='123456789012.dkr.ecr.us-east-1.amazonaws.com/akirmak-smprocessing:latest',
                role=role,
                instance_count=1,
                instance_type='ml.m5.xlarge')

In [49]:
# script_processor.run(code='inference.py',
#                      inputs=[ProcessingInput(
#                         source='s3://path/to/my/input-data.csv',
#                         destination='/opt/ml/processing/input')],
#                      outputs=[ProcessingOutput(source='/opt/ml/processing/output/train'),
#                                ProcessingOutput(source='/opt/ml/processing/output/validation'),
#                                ProcessingOutput(source='/opt/ml/processing/output/test')])

In [ ]:
script_processor.run(
    code="inference.py",
    inputs=[
        ProcessingInput(source="s3://akirmak-r/text_classification.yaml", destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train_data", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/test"),
    ],
    arguments=["--train-test-split-ratio", "0.2", "--dummy-argument", "hello NLP article team"],
)

inference_job_description = script_processor.jobs[-1].describe()

In [ ]:
from sagemaker.pytorch import PyTorchModel
!aws s3 ls {model_url}
from nlc.jobutil import get_role
role = get_role()

pytorch_model = PyTorchModel(model_data=model_url,
                             role=role,
                             py_version='py3',
                             framework_version='1.8',
                             source_dir='.',
                             entry_point='inference.py')

# Pipelines

https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-pipelines/tabular/abalone_build_train_deploy/sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb

In [51]:
input_data_uri = 's3://prj-ml/banking.csv'

In [52]:

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)


processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.m5.xlarge"
)
training_instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)
input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)

In [53]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep


step_process = ProcessingStep(
    name="hba-SMPipelinesTestBed-Process",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
    ],
    code="processing.py",
)


In [54]:
# script_eval = 'inference.py'

In [55]:
from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)

step_eval = ProcessingStep(
    name="hba-evaluation-smprocessing",
    processor=script_processor,
    inputs=[
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="inference.py",
    property_files=[evaluation_report]
)

In [56]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = f"hbaSMPipelineTestBed"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        training_instance_type,
        model_approval_status,
        input_data,
    ],
    steps=[step_process, step_eval],
)

In [57]:
import json


definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ModelApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'PendingManualApproval'},
  {'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://prj-ml/banking.csv'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'hba-SMPipelinesTestBed-Process',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': 'ml.m5.xlarge',
      'InstanceCount': 1,
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3

In [58]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:1234567890:pipeline/hbasmpipelinetestbed',
 'ResponseMetadata': {'RequestId': '49402554-bb10-4192-86af-19da3defef45',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '49402554-bb10-4192-86af-19da3defef45',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '88',
   'date': 'Sun, 29 Aug 2021 14:33:11 GMT'},
  'RetryAttempts': 0}}

In [59]:
execution = pipeline.start()

In [61]:
execution.describe()


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:1234567890:pipeline/hbasmpipelinetestbed',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:1234567890:pipeline/hbasmpipelinetestbed/execution/j5alfiumea9y',
 'PipelineExecutionDisplayName': 'execution-1630247594493',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'hbasmpipelinetestbed',
  'TrialName': 'j5alfiumea9y'},
 'CreationTime': datetime.datetime(2021, 8, 29, 14, 33, 14, 431000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 8, 29, 14, 33, 14, 431000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:1234567890:user-profile/d-1bf2r1y78nt5/akirmak-amazon-com-17e',
  'UserProfileName': 'akirmak-amazon-com-17e',
  'DomainId': 'd-1bf2r1y78nt5'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:1234567890:user-profile/d-1bf2r1y78nt5/akirmak-amazon-com-17e',
  'UserProfileName': 'akirmak-amazon-com-17e',
  'DomainId': 'd-1bf2r1y7

In [ ]:
execution.wait()

In [ ]:
execution.list_steps()


In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer


viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    display(viz.show(pipeline_execution_step=execution_step))
    time.sleep(5)